# Redes generativas

Redes Neuronais Recorrentes (RNNs) e suas variantes com células controladas, como Células de Memória de Longo e Curto Prazo (LSTMs) e Unidades Recorrentes Controladas (GRUs), proporcionaram um mecanismo para modelagem de linguagem, ou seja, elas podem aprender a ordem das palavras e fornecer previsões para a próxima palavra em uma sequência. Isso permite que utilizemos RNNs para **tarefas generativas**, como geração de texto comum, tradução automática e até mesmo legendagem de imagens.

Na arquitetura de RNN que discutimos na unidade anterior, cada unidade RNN produzia o próximo estado oculto como saída. No entanto, também podemos adicionar outra saída a cada unidade recorrente, o que nos permitiria gerar uma **sequência** (que tem o mesmo comprimento da sequência original). Além disso, podemos usar unidades RNN que não aceitam uma entrada em cada etapa, mas apenas recebem um vetor de estado inicial e, em seguida, produzem uma sequência de saídas.

Neste notebook, vamos focar em modelos generativos simples que nos ajudam a gerar texto. Para simplificar, vamos construir uma **rede ao nível de caracteres**, que gera texto letra por letra. Durante o treino, precisamos pegar um corpus de texto e dividi-lo em sequências de letras.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Construir um vocabulário de caracteres

Para construir uma rede generativa a nível de caracteres, precisamos dividir o texto em caracteres individuais em vez de palavras. A camada `TextVectorization` que temos usado anteriormente não consegue fazer isso, por isso temos duas opções:

* Carregar o texto manualmente e fazer a tokenização 'à mão', como neste [exemplo oficial do Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Usar a classe `Tokenizer` para a tokenização a nível de caracteres.

Vamos optar pela segunda opção. A `Tokenizer` também pode ser usada para tokenizar em palavras, por isso deve ser possível alternar facilmente entre a tokenização a nível de caracteres e a nível de palavras.

Para realizar a tokenização a nível de caracteres, precisamos passar o parâmetro `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Também queremos usar um token especial para indicar **fim de sequência**, que chamaremos de `<eos>`. Vamos adicioná-lo manualmente ao vocabulário:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Treinar uma RNN generativa para criar títulos

A forma como iremos treinar a RNN para gerar títulos de notícias é a seguinte. Em cada passo, iremos pegar um título, que será alimentado numa RNN, e para cada caractere de entrada pediremos à rede que gere o próximo caractere de saída:

![Imagem mostrando um exemplo de geração de RNN da palavra 'HELLO'.](../../../../../lessons/5-NLP/17-GenerativeNetworks/images/rnn-generate.png)

Para o último caractere da nossa sequência, pediremos à rede que gere o token `<eos>`.

A principal diferença da RNN generativa que estamos a usar aqui é que iremos utilizar a saída de cada passo da RNN, e não apenas da célula final. Isto pode ser conseguido ao especificar o parâmetro `return_sequences` na célula da RNN.

Assim, durante o treino, a entrada para a rede será uma sequência de caracteres codificados de um determinado comprimento, e a saída será uma sequência do mesmo comprimento, mas deslocada por um elemento e terminada com `<eos>`. O minibatch consistirá em várias dessas sequências, e será necessário usar **padding** para alinhar todas as sequências.

Vamos criar funções que irão transformar o conjunto de dados para nós. Como queremos adicionar padding às sequências ao nível do minibatch, primeiro agruparemos o conjunto de dados ao chamar `.batch()`, e depois aplicaremos `map` para realizar a transformação. Portanto, a função de transformação irá receber um minibatch inteiro como parâmetro:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Algumas coisas importantes que fazemos aqui:  
* Primeiro extraímos o texto real do tensor de strings  
* `text_to_sequences` converte a lista de strings numa lista de tensores inteiros  
* `pad_sequences` preenche esses tensores até ao seu comprimento máximo  
* Finalmente, fazemos a codificação one-hot de todos os caracteres, além de realizar o deslocamento e a adição de `<eos>`. Em breve veremos por que precisamos de caracteres codificados em one-hot  

No entanto, esta função é **Pythonic**, ou seja, não pode ser automaticamente traduzida para o grafo computacional do Tensorflow. Iremos obter erros se tentarmos usar esta função diretamente na função `Dataset.map`. Precisamos encapsular esta chamada Pythonic utilizando o wrapper `py_function`:  


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Nota**: Diferenciar entre funções de transformação Pythonic e Tensorflow pode parecer um pouco complexo, e pode estar a questionar-se por que não transformamos o conjunto de dados usando funções padrão de Python antes de passá-lo para `fit`. Embora isso definitivamente possa ser feito, usar `Dataset.map` tem uma grande vantagem, porque o pipeline de transformação de dados é executado usando o grafo computacional do Tensorflow, que aproveita os cálculos na GPU e minimiza a necessidade de transferir dados entre CPU/GPU.

Agora podemos construir a nossa rede geradora e começar o treino. Pode ser baseada em qualquer célula recorrente que discutimos na unidade anterior (simples, LSTM ou GRU). No nosso exemplo, usaremos LSTM.

Como a rede recebe caracteres como entrada e o tamanho do vocabulário é relativamente pequeno, não precisamos de uma camada de embedding; a entrada codificada em one-hot pode ir diretamente para a célula LSTM. A camada de saída será um classificador `Dense` que converterá a saída do LSTM em números de tokens codificados em one-hot.

Além disso, como estamos a lidar com sequências de comprimento variável, podemos usar a camada `Masking` para criar uma máscara que ignorará a parte preenchida da string. Isto não é estritamente necessário, porque não estamos muito interessados em tudo o que vai além do token `<eos>`, mas usaremos esta camada para ganhar alguma experiência com este tipo de camada. O `input_shape` será `(None, vocab_size)`, onde `None` indica a sequência de comprimento variável, e o formato de saída será `(None, vocab_size)` também, como pode ver no `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Gerar resultados

Agora que treinámos o modelo, queremos utilizá-lo para gerar alguns resultados. Antes de mais, precisamos de uma forma de decodificar texto representado por uma sequência de números de tokens. Para isso, poderíamos usar a função `tokenizer.sequences_to_texts`; no entanto, esta não funciona bem com tokenização a nível de caracteres. Por isso, vamos pegar num dicionário de tokens do tokenizer (chamado `word_index`), construir um mapa inverso e escrever a nossa própria função de decodificação:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Agora, vamos começar a geração. Iniciaremos com uma string `start`, codificá-la-emos numa sequência `inp`, e em cada passo chamaremos a nossa rede para inferir o próximo carácter.

O output da rede `out` é um vetor de elementos `vocab_size` que representam as probabilidades de cada token, e podemos encontrar o número do token mais provável utilizando `argmax`. Em seguida, adicionamos este carácter à lista de tokens gerados e continuamos com a geração. Este processo de gerar um carácter é repetido `size` vezes para gerar o número necessário de caracteres, e terminamos antecipadamente quando o `eos_token` é encontrado.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Amostragem de saída durante o treino

Como não temos métricas úteis como *precisão*, a única forma de verificar se o nosso modelo está a melhorar é através da **amostragem** de strings geradas durante o treino. Para isso, utilizaremos **callbacks**, ou seja, funções que podemos passar para a função `fit`, e que serão chamadas periodicamente durante o treino.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Este exemplo já gera um texto bastante bom, mas pode ser melhorado de várias maneiras:

* **Mais texto**. Utilizámos apenas títulos para a nossa tarefa, mas pode querer experimentar com texto completo. Lembre-se de que as RNNs não lidam muito bem com sequências longas, por isso faz sentido dividi-las em frases mais curtas ou treinar sempre com uma sequência fixa de comprimento pré-definido `num_chars` (por exemplo, 256). Pode tentar alterar o exemplo acima para essa arquitetura, usando o [tutorial oficial do Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) como inspiração.

* **LSTM com várias camadas**. Faz sentido experimentar 2 ou 3 camadas de células LSTM. Como mencionámos na unidade anterior, cada camada de LSTM extrai certos padrões do texto, e no caso de um gerador a nível de caracteres, podemos esperar que o nível inferior do LSTM seja responsável por extrair sílabas, e os níveis superiores - palavras e combinações de palavras. Isto pode ser implementado facilmente passando o parâmetro número-de-camadas ao construtor do LSTM.

* Também pode querer experimentar com **unidades GRU** e ver quais produzem melhores resultados, bem como com **diferentes tamanhos de camadas ocultas**. Uma camada oculta demasiado grande pode resultar em overfitting (por exemplo, a rede aprenderá o texto exato), e um tamanho menor pode não produzir bons resultados.


## Geração de texto suave e temperatura

Na definição anterior de `generate`, estávamos sempre a escolher o carácter com a maior probabilidade como o próximo carácter no texto gerado. Isto resultava no facto de o texto frequentemente "ciclar" entre as mesmas sequências de caracteres repetidamente, como neste exemplo:
```
today of the second the company and a second the company ...
```

No entanto, se analisarmos a distribuição de probabilidades para o próximo carácter, pode acontecer que a diferença entre algumas das probabilidades mais altas não seja muito grande, por exemplo, um carácter pode ter uma probabilidade de 0.2, outro de 0.19, etc. Por exemplo, ao procurar o próximo carácter na sequência '*play*', o próximo carácter pode ser igualmente um espaço ou **e** (como na palavra *player*).

Isto leva-nos à conclusão de que nem sempre é "justo" selecionar o carácter com maior probabilidade, porque escolher o segundo mais provável ainda pode levar-nos a um texto significativo. É mais sensato **amostrar** caracteres a partir da distribuição de probabilidades fornecida pela saída da rede.

Esta amostragem pode ser feita utilizando a função `np.multinomial`, que implementa a chamada **distribuição multinomial**. Uma função que implementa esta geração de texto **suave** está definida abaixo:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Introduzimos mais um parâmetro chamado **temperatura**, que é usado para indicar o quão rigorosamente devemos aderir à maior probabilidade. Se a temperatura for 1.0, fazemos uma amostragem multinomial justa, e quando a temperatura vai para infinito - todas as probabilidades tornam-se iguais, e selecionamos aleatoriamente o próximo caractere. No exemplo abaixo, podemos observar que o texto torna-se sem sentido quando aumentamos demasiado a temperatura, e assemelha-se a um texto "ciclado" gerado rigidamente quando se aproxima de 0.



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante notar que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
